# Address

In [1]:
# os.listdir('../address')

In [2]:
# for file in os.listdir('../address')[1:]:
#     if file.endswith('txt'):
#         city = pd.read_csv(f"../address/{file}", sep = '|', header = None, encoding = 'euc-kr')
#         city.columns = txt_headers
#         city = city.drop_duplicates(subset = '읍면동명', ignore_index = True)
#         for i in range(i, len(city), 100):
#             city[i:i+100].to_csv(f"../address/{file[:-4]}_{i//100}.csv", index = False)

# json csv 저장

In [3]:
# import pandas as pd
# import os

# for file in os.listdir('./Yogiyo'):
#     if file.endswith('.json'):
#         pd.DataFrame(open(f'./Yogiyo/{file}')).to_csv(f"./Yogiyo/{file[:-5]}.csv", index = False)
#     else:
#         print("Dk")

# Curation

In [4]:
import datetime
import math
import os, sys
import time
import pymysql
import numpy as np
import pandas as pd
from tqdm import tqdm
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))

# db management libraries
import pymysql
from dbconfig import Insert, Upsert, reduce_mem_usage
from controller import MysqlController

# similarity, prediction
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
mysql_controller = MysqlController('mealgen-db-real-cluster.cluster-ch1v7vb7kdnw.ap-northeast-2.rds.amazonaws.com', 3306, 'mealgen', 'altlrdmltleo1!', 'mealgen')

In [6]:
df = pd.read_sql("SELECT v.nickname, v.user_id, v.menu_id, r.sigungu, v.restaurant_id, v.like_dislike FROM reviews as v JOIN restaurant_info as r ON v.restaurant_id = r.restaurant_id;", mysql_controller.conn)
# df.head()

In [7]:
df = reduce_mem_usage(df)
df.info()

Before Reduce : 20.894 MB
After Reduce :  12.624 MB ( 60.42% of the initial size)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456443 entries, 0 to 456442
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   nickname       456443 non-null  object
 1   user_id        456443 non-null  uint32
 2   menu_id        456443 non-null  int32 
 3   sigungu        451587 non-null  object
 4   restaurant_id  456443 non-null  uint32
 5   like_dislike   456443 non-null  uint8 
dtypes: int32(1), object(2), uint32(2), uint8(1)
memory usage: 12.6+ MB


In [8]:
dict(df.iloc[1])

{'nickname': 'ch**',
 'user_id': 43937495,
 'menu_id': 46856583,
 'sigungu': ' 관악구',
 'restaurant_id': 139,
 'like_dislike': 1}

In [9]:
menu_dict = dict(df[['menu_id', 'restaurant_id']].drop_duplicates().values)

In [10]:
df['user'] = df['sigungu'] + df['nickname']

In [11]:
df.head()

,nickname,user_id,menu_id,sigungu,restaurant_id,like_dislike,user
0,sh**,43976662,2099967,관악구,139,1,관악구sh**
1,ch**,43937495,46856583,관악구,139,1,관악구ch**
2,sh**,43409231,46856578,관악구,139,1,관악구sh**
3,xf**,41248020,46856578,관악구,139,1,관악구xf**
4,yo**,41115610,2099914,관악구,139,1,관악구yo**


In [12]:
new = pd.DataFrame(columns = df.menu_id.unique())
new.head()

,2099967,46856583,46856578,2099914,2099960,2099925,4998071,2099961,2099964,2099955,...,119028095,8523455,6197208,6197165,6197211,6197213,12482995,42069067,42069073,42069017


## Cosine Similarity

In [13]:
len(df) // 10000

45

In [14]:
d3 = df[:10000].pivot(columns = 'menu_id', index = 'user_id', values = 'like_dislike')
d3.head()

menu_id,-1,5800,11067,11988,16964,16967,16968,16970,40025,40029,...,178626144,178626303,178626308,178626330,178626334,178626342,179253103,179253106,179253110,179622682
user_id,,,,,,,,,,,,,,,,,,,,,
844735,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
len(d3.columns)

784

In [16]:
d3 = d3.fillna(0)

In [17]:
# rated_df = df.pivot(columns = 'menu_id', index = 'user_id', values = 'like_dislike')
# del df

In [18]:
# rated_df = rated_df.fillna(0)

In [19]:
rated_df = d3

In [20]:
# 만들어진 pivot table로 User간 코사인 유사도 계산
cos_sim = pd.DataFrame(cosine_similarity(d3), index = d3.index, columns= d3.index) # 5만개 164초 # 3만개 25초 # 1만개 1.3

In [21]:
cos_sim.head()

user_id,844735,844763,848248,851132,851543,854439,855285,862896,864195,866283,...,46703788,46729365,46744989,46755217,46767422,46770943,46777193,46791199,46798438,46804972
user_id,,,,,,,,,,,,,,,,,,,,,
844735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
844763,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
848248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
851132,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
851543,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
compat_df = pd.DataFrame(cos_sim.unstack())
del cos_sim
compat_df.index.names = ['user_id', 'target_user_id']
compat_df = compat_df.reset_index()
compat_df.columns = ['user_id', 'target_user_id', 'expect_rate']

In [23]:
# compat_df['update_yn'] = 1
# # User간 궁합점수가 0% ~ 100%로 나오기 때문에 이에 맞게 변환
compat_df['expect_rate'] = compat_df['expect_rate'].apply(lambda x: ((x+1)/2)*100)
# 메모리 줄이기
compat_df = reduce_mem_usage(compat_df) # 37초

Before Reduce : 2288.818 MB
After Reduce :  858.307 MB ( 37.50% of the initial size)


In [24]:
compat_df # user_comp 에 update

,user_id,target_user_id,expect_rate
0,844735,844735,50
1,844735,844763,50
2,844735,848248,50
3,844735,851132,50
4,844735,851543,50
...,...,...,...
99999995,46804972,46770943,50
99999996,46804972,46777193,50
99999997,46804972,46791199,50
99999998,46804972,46798438,50


In [25]:
line = dict(compat_df.iloc[1])
line
",".join(['%s']*len(line))

line.values()

dict_values([844735, 844763, 50])

## SVD

In [26]:
# 점수 범위가 0,1
reader = Reader(rating_scale = (0, 1))
# 사용자의 메뉴에 대한 평점이므로, df에서 필요한 부분만 가져와서 데이터셋으로 만들기
dataset = Dataset.load_from_df(df[['user_id', 'menu_id', 'like_dislike']], reader)
# 전체 데이터를 train set으로 활용
# trainset = dataset.build_full_trainset()
trainset, testset = train_test_split(dataset, test_size = .25)

# SVD 알고리즘 적용
svd = SVD()
# train
svd.fit(trainset)

In [27]:
# dataset에서 testset만들어 prediction확인
pred = svd.test(testset)

In [28]:
#rmse로 측정
accuracy.rmse(pred)

RMSE: 0.2706


0.27063799685455403

## Account Predict table UPDATE

In [29]:
# update하기 위해 numpy형태로 변환
# return되는 형식: [account_id, menu_id]
def make_predict_list(df, uid):
    res_list = df['menu_id'].unique()
#     이 부분은 사용자가 평가한 메뉴는 예측에서 제외할 때(현재는 다 같이 포함해서 예측)
#     past_res_list = df[(df['account_id'] == int(uid))]['menu_id'].unique()
#     pred_res_list = list(set(res_list) - set(past_res_list))
    pred_res_list = np.c_[[uid] * len(res_list), res_list]
    return pred_res_list

In [30]:
def make_recommendation(algo, uid, pred_res_list):
    rate_list = []
    for i in pred_res_list:
        # 위에서 만든 [account_id, menu_id]를 받아서 이에 대한 예상 점수 계산
        rate = algo.predict(i[0], i[1]).est
        rate_list.append(rate)
    rate_list = np.array(rate_list)
    pred_res_list = np.c_[pred_res_list, rate_list]
    # [account_id, menu_id, rate]를 df로 변환
    recomm_df = pd.DataFrame(pred_res_list, columns = ['user_id', 'menu_id', 'predict'])
    return recomm_df

In [31]:
recomm_df = pd.DataFrame(columns = ['user_id', 'menu_id', 'predict'])
for uid in tqdm(df['user_id'].unique()[:10]):
    pred_res_list = make_predict_list(df, uid)
    temp = make_recommendation(svd, uid, pred_res_list)
    recomm_df = pd.concat([recomm_df, temp]).reset_index(drop = True)

100%|██████████| 10/10 [00:01<00:00,  5.14it/s]


In [32]:
# 예상점수 높은 순 정렬
recomm_df[recomm_df['user_id'] == 43976662].sort_values(['predict'], ascending = False)

,user_id,menu_id,predict
11239,43976662.0,519664.0,1.000000
3020,43976662.0,1181182.0,1.000000
3014,43976662.0,3645861.0,1.000000
5672,43976662.0,74784712.0,1.000000
4338,43976662.0,1350754.0,1.000000
...,...,...,...
7469,43976662.0,14108.0,0.709920
112,43976662.0,52681.0,0.701073
1869,43976662.0,1028676.0,0.690451
7452,43976662.0,2274632.0,0.621884


In [33]:
# # 만들어뒀던 menu-restaurant dictionary를 dataframe에 추가
recomm_df['restaurant_id'] = recomm_df['menu_id'].apply(lambda x: menu_dict[x])
# recomm_df['menu'] = recomm_df['menu_id'].apply(lambda x: menu_dict[x]) 
# # db에서 기존에 있던 데이터와 구분하기 위해 column추가
# recomm_df['update_yn'] = 1

In [34]:

recomm_df = recomm_df.reindex(columns = ['user_id', 'menu_id', 'restaurant_id', 'predict'])
# recomm_df['update_yn'] = 1

In [35]:
# recomm_df.rename(columns = {'like': 'predict'})

In [38]:
df.head()

,nickname,user_id,menu_id,sigungu,restaurant_id,like_dislike,user
0,sh**,43976662,2099967,관악구,139,1,관악구sh**
1,ch**,43937495,46856583,관악구,139,1,관악구ch**
2,sh**,43409231,46856578,관악구,139,1,관악구sh**
3,xf**,41248020,46856578,관악구,139,1,관악구xf**
4,yo**,41115610,2099914,관악구,139,1,관악구yo**


In [36]:
recomm_df = reduce_mem_usage(recomm_df)
recomm_df # 이거가 account_predict에 update됨

Before Reduce : 11.050 MB
After Reduce :  5.525 MB ( 50.00% of the initial size)


,user_id,menu_id,restaurant_id,predict
0,43976662,2099967,139,0.969734
1,43976662,46856583,139,1.000000
2,43976662,46856578,139,0.971271
3,43976662,2099914,139,0.999803
4,43976662,2099960,139,1.000000
...,...,...,...,...
362065,39794491,6197213,17933,0.699911
362066,39794491,12482995,17933,0.852937
362067,39794491,42069067,17984,0.957613
362068,39794491,42069073,17984,0.909918


In [37]:
Upsert(mysql_controller, table_name = 'user_predict', line = dict(recomm_df.iloc[0]))

In [39]:
m_q = "SELECT DISTINCT menu_id, name FROM menu_info;"
mysql_controller.curs.execute(m_q)
menu = dict(mysql_controller.curs.fetchall())

In [40]:
r_q = "SELECT DISTINCT restaurant_id, name FROM restaurant_info;"
mysql_controller.curs.execute(r_q)
res = dict(mysql_controller.curs.fetchall())

In [49]:
recomm_df

,user_id,menu_id,restaurant_id,predict
0,43976662,2099967,139,0.969734
1,43976662,46856583,139,1.000000
2,43976662,46856578,139,0.971271
3,43976662,2099914,139,0.999803
4,43976662,2099960,139,1.000000
...,...,...,...,...
362065,39794491,6197213,17933,0.699911
362066,39794491,12482995,17933,0.852937
362067,39794491,42069067,17984,0.957613
362068,39794491,42069073,17984,0.909918


In [50]:
recomm_df[recomm_df.menu_id == -1]

,user_id,menu_id,restaurant_id,predict
41,43976662,-1,4729,0.951682
36248,43937495,-1,4729,0.922746
72455,43409231,-1,4729,0.943868
108662,41248020,-1,4729,0.945582
144869,41115610,-1,4729,0.975614
181076,41033000,-1,4729,0.951682
217283,40972126,-1,4729,0.933269
253490,40008655,-1,4729,0.941065
289697,39864635,-1,4729,0.959313
325904,39794491,-1,4729,0.960379


In [48]:
rc_df = recomm_df.drop(recomm_df[recomm_df.menu_id == -1].index)
rc_df

,user_id,menu_id,restaurant_id,predict
0,43976662,2099967,139,0.969734
1,43976662,46856583,139,1.000000
2,43976662,46856578,139,0.971271
3,43976662,2099914,139,0.999803
4,43976662,2099960,139,1.000000
...,...,...,...,...
362065,39794491,6197213,17933,0.699911
362066,39794491,12482995,17933,0.852937
362067,39794491,42069067,17984,0.957613
362068,39794491,42069073,17984,0.909918


In [53]:
recomm_df = recomm_df.drop(recomm_df[recomm_df.menu_id == -1].index)
recomm_df['menu'] = recomm_df['menu_id'].apply(lambda x: menu[x])
recomm_df['restaurant'] = recomm_df['restaurant_id'].apply(lambda x: res[x])
# 순서 정렬
recomm_df = recomm_df.reindex(columns = ['user_id', 'menu_id', 'menu', 'restaurant_id', 'restaurant', 'predict'])
recomm_df = reduce_mem_usage(recomm_df)
for i in tqdm(range(len(recomm_df))):
    Upsert(mysql_controller, table_name='user_predict', line = dict(recomm_df.iloc[i]))

  0%|          | 4/362060 [00:00<2:47:43, 35.98it/s]

Before Reduce : 23.811 MB
After Reduce :  23.811 MB ( 100.00% of the initial size)


  4%|▍         | 14188/362060 [06:11<2:31:00, 38.39it/s]